# Extra Credit

For upto an additional 15 extra credit points on the midterm exam, you can write another function that applies additional criteria on top of what the midterm already did.  To earn the addition points, you need write a function called `summarize_results(patients)` to take the output of your  `process_people()` function to apply a new business rule and summarize the results.

* Your function should take the dictionary created by `process_people()` as it's one input parameter: `patients`.
* If the PRIEST probably is less than 10%, change the `hospital` and `address` to both have a value of `'Home'` indicating patients with a lower probability of an adverse outcome can just go home rather than be admitted to a hospital.
* Summarize those results and count how many patients are now being sent to each location.  Your result should have the following format:
```
{
    "Home": 123,
    "Hospital A": 10,
    "Hospital B": 20,
    "Hospital C": 30
}
```

As always, your function needs good docstrings; but does not require any doctests in this case.

In [3]:
import requests
import json

def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP, temperature, alertness, inspired_oxygen, performance_status):
    """
    (str, int, int, float, int, int, float, str, str, str) -> (float)
    The priest function initially determines an individual values of COVID-19 severity based on the above given factors.
    The algorithm used is available on the MDCalc website.
    Based on the severity, we may then predict the probability that the patient would have any adverse effects within the next 30 days.

    >>> priest("female", 58, 18, 0.94, 88, 112, 38.3, 'Alert', 'air', 'Unrestricted normal activity')
    0.03
    >>> priest ("FemalE", 78, 15, 0.92, 100, 200, 31.0, 'confused or not alert', 'supplemental OxYGEN', 'limited strenuous activity, can do light activity')
    0.49
    >>> priest("MaLe", 85, 19, 0.90, 80, 115, 38.2, 'CONFUSED OR not ALERT','SUPPLEMENTAL OxygEN','LIMITED activity, can SELF-CARE')
    0.55
    >>> priest('FEMALE', 66, 19, 0.96, 135, 115, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care')
    0.34
    >>> priest('female', 51, 23.0, 0.94, 135, 115, 38.1, 'ALERT', 'AIR', 'limited strenuous activity, can do light activity')
    0.29
    """

    score = 0
    sex = sex.lower()
    if sex == "male":
        score += 1
    elif sex != "female":
        return None

    if 16 <= age <= 49:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4

    if respiratory_rate < 9:
        score += 3
    elif 9 <= respiratory_rate <= 11:
        score += 1
    elif 12 <= respiratory_rate <= 20:
        score += 0
    elif 21 <= respiratory_rate <= 24:
        score += 2
    elif respiratory_rate > 24:
        score += 3

    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif oxygen_saturation >= 0.92 <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3

    if systolic_BP < 91:
        score += 3
    elif 91 <= systolic_BP <= 100:
        score += 2
    elif 101 <= systolic_BP <= 110:
        score += 1
    elif 111 <= systolic_BP <= 219:
        score += 0
    elif systolic_BP > 219:
        score += 3

    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36.0:
        score += 1
    elif 36.1 <= temperature <= 38.0:
        score += 0
    elif 38.1 <= temperature <= 39.0:
        score += 1
    elif temperature > 39.0:
        score += 2

    alertness = alertness.lower()
    if alertness == "alert":
        score += 0
    elif alertness == "confused or not alert":
        score += 3

    if inspired_oxygen.lower() == "air":
        score += 0
    elif inspired_oxygen.lower() == "supplemental oxygen":
        score += 2

    if performance_status.lower() == "unrestricted normal activity":
        score += 0
    elif performance_status.lower() == "limited strenuous activity, can do light activity":
        score += 1
    elif performance_status.lower() == "limited activity, can self-care":
        score += 2
    elif performance_status.lower() == "limited self-care":
        score += 3
    elif performance_status.lower() == "bed/chair bound, no self-care":
        score += 4

    risk = 0.0
    if 0 <= score <= 1:
        risk = 0.01
    elif 2 <= score <= 3:
        risk = 0.02
    elif score == 4:
        risk = 0.03
    elif score == 5:
        risk = 0.09
    elif score == 6:
        risk = 0.15
    elif score == 7:
        risk = 0.18
    elif score == 8:
        risk = 0.22
    elif score == 9:
        risk = 0.26
    elif score == 10:
        risk = 0.29
    elif score == 11:
        risk = 0.34
    elif score == 12:
        risk = 0.38
    elif score == 13:
        risk = 0.46
    elif score == 14:
        risk = 0.47
    elif score == 15:
        risk = 0.49
    elif score == 16:
        risk = 0.55
    elif 17 <= score <= 25:
        risk = 0.59
    elif score >= 26:
        risk = 0.99

    return risk

def find_hospital(age, sex, risk):
    """Returns the hospital name give the age, sex, and risk factor.
    Invalid value for age returns None:
    >>> find_hospital(-1, "female", 0.4)

    Invalid value for sex returns None:
    >>> find_hospital(30, "xxxx", 0.5)

    Invalid value for risk returns None:
    >>> find_hospital(42, "male", 1.1)

    >>> find_hospital(40, "male", 0.1)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(35, "female", 0.7)
    'Emory Dunwoody Medical Center'
    """

    url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={0}&sex={1}&risk_pct={2}"

    if sex.lower() not in ["male", "female"]:
        return None
    if age < 0:
        return None
    if risk is None or risk < 0.0 or risk > 1.0:
        return None
    response = requests.get(url.format(age, sex.lower(), risk))
    if response.status_code == 200:
        return response.json()["hospital"]
    else:
        return None

def get_address_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    # f = open('ga_hospitals.json')
    # data = json.load(f)
    # return data

api_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
hospital_addresses = get_address_from_url(api_url)

def get_address(hospital_name):
    """
    :param hospital_name:
    :return: hospital_address
    >>> get_address("")
    >>> get_address("unknown hospital")
    >>> get_address("Emory Dunwoody Medical Center")
    '4500 NORTH SHALLOWFORD ROAD'
    """
    if hospital_name is None:
        return None

    hospital_name = hospital_name.upper()
    if hospital_name in hospital_addresses:
        return hospital_addresses[hospital_name]["ADDRESS"]
    else:
        return None

def process_people(url):
    # with open('people.csv') as file:
    #     content = file.readlines()
    people = {}
    response = requests.get(url)
    for row in response.text.split("\n")[1:]:
        fields = row.strip().split("|")
        if(len(fields) == 11):
            patient_id = fields[0]
            (sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP,
            temperature, alertness, inspired_oxygen, performance_status) = \
                (fields[1], int(fields[2]), float(fields[3]), float(fields[4]),
                float(fields[5]), float(fields[6]), float(fields[7]), fields[8], fields[9], fields[10])
            risk = priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP,
                        temperature, alertness, inspired_oxygen, performance_status)
            hospital = find_hospital(age, sex.lower(), risk)
            address = get_address(hospital)
            people[patient_id] = [sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP,
                              temperature, alertness, inspired_oxygen, performance_status, risk, hospital, address]

    return people

def summarize_results(patients):
    """
    This function prepares the summary of all the hospitals with the patient counts
    :param patients: The patient dictionary
    :return: the summary of number patients that are going to each hospital
    """
    risk_index = 10
    hospital_index = 11
    address_index = 12
    results = {"Home": 0}
    for patient in patients.values():
        if patient[risk_index] < 0.10:
            patient[hospital_index] = "Home"
            patient[address_index] = "Home"
            results["Home"] += 1

        hospital = patient[hospital_index]

        if hospital in results:
            results[hospital] += 1
        else:
            results[hospital] = 0

    return results

In [4]:
people = process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")
summary = summarize_results(people)
print(summary)

{'Home': 48, 'Select Specialty Hospital - Northeast Atlanta': 31, 'Wesley Woods Geriatric Hospital': 19, 'Southwest Hospital and Medical Center': 15, 'Childrens Healthcare of Atlanta at Scottish Rite': 0, 'Hughes Spalding Childrens Hospital': 0, 'Emory Dunwoody Medical Center': 4}
